In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
import sys
import datetime
import time

In [ ]:
driver = webdriver.Chrome(executable_path = r'C:\Users\Administrator\Downloads\chromedriver.exe')

In [ ]:
df1 = pd.read_excel(r'Philips Top Products.xlsx',header = 0,engine = 'openpyxl')

In [ ]:
# Need to get catalog images
# driver.get('https://www.flipkart.com/product/p/itme?pid=MOBGYFAXFPHUPEE5')
def getCatalogContent(driver):
    
    imgs = driver.find_elements_by_css_selector('div>ul>li>div>div[style*=background-image]')
    numVids = len(driver.find_elements_by_css_selector('div>ul>li>div>div:nth-child(3)'))
    imgList = []
    for i in imgs:
        url_ = i.value_of_css_property('background-image').strip('url("")').replace('128/','416/',2)
        imgList.append(url_)
    return imgList,numVids


In [ ]:
# Need to get all the remaining info on the webPage including features, description and Product title



In [ ]:
# product_title = driver.find_element_by_css_selector('div>div>div>h1').text
def getTechnicalDetails(driver):
    details_fs = driver.find_elements_by_css_selector('table')
    df_prodTabularDetails = pd.DataFrame()
    productDetails = {}
    for tl in details_fs:
        table_content = tl.get_attribute('innerHTML')
        # pd.read_html(table_content)
        df = pd.read_html('<table>' + table_content + '</table>')[0]
        df_prodTabularDetails = df_prodTabularDetails.append(df,ignore_index=True)
    productDetails = df_prodTabularDetails.set_index(0)[1].to_dict()
    try:
        description = driver.find_elements_by_xpath('//div[contains(text(),"Description")]/following-sibling::div')
        driver.execute_script("arguments[0].setAttribute('style','max-height:None;overflow:None;')", description[0])
        productDetails['Description_product'] = description[0].text
    except:
        pass
    productDetails['timestamp'] = str(datetime.datetime.now())
    return productDetails



In [ ]:
def getFeatureDetails(driver):
    feature_button = driver.find_elements_by_xpath("//button[text()='View all features']")
    if(len(feature_button)!=0):
        feature_button[0].click()
    details_elements = driver.find_elements_by_css_selector('div.col-12-12 div:nth-child(2)>p')

    featureDetails = {}
    for ftrIdx in range(len(details_elements)):
        featureDetails['feature_' + str(ftrIdx)] = details_elements[ftrIdx].text
    featureDetails['timestamp'] = str(datetime.datetime.now())
    return featureDetails


In [110]:
driver.get('https://www.flipkart.com/product/p/itme?pid=IRNFZV2YUBUZ3PTZ')

In [ ]:
def getHlEmiDetails(driver):
    OtherDetails = driver.find_elements_by_css_selector('div.col-6-12')
    hl_emi_details = {}
    for hl_emi_idx in range(len(OtherDetails)):
        _detail = OtherDetails[hl_emi_idx].text
        detail_pair = _detail.split('\n',maxsplit = 1)
        if(_detail!='' and len(detail_pair)>1):
            hl_emi_details[detail_pair[0]] = detail_pair[1]
    hl_emi_details['timestamp'] = str(datetime.datetime.now())
    return hl_emi_details

In [ ]:
def getASINData(asin,timeout,locator,driver):
    link = 'https://www.flipkart.com/product/p/itme?pid=' + str(asin)

    driver.get(link)
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, locator)))
    product_title = driver.find_element_by_css_selector('div>div>div>h1').text
    img_list,num_vids = getCatalogContent(driver)
    productDetails = getTechnicalDetails(driver)
    featureDetails = getFeatureDetails(driver)
    hl_emi_details =  getHlEmiDetails(driver)
    
    final_res = {}
    final_res['product title'] = product_title
    final_res['product details'] = productDetails
    final_res['feature details'] = featureDetails
    final_res['hl_emi details'] = hl_emi_details
    final_res['images'] = img_list
    final_res['num vids'] = num_vids
    final_res['timestamp'] = str(datetime.datetime.now())
    return final_res

In [ ]:
def getData(asin_list,asin_start_index,timeout,locator,driver):
    asin_page = {}
    current_asin_index = 0
    for asin_idx in tqdm(range(asin_start_index,len(asin_list))):
        try:
            asin = asin_list[asin_idx]
            asin_page[asin] = getASINData(asin,timeout,locator,driver)
            current_asin_index = asin_idx
        except KeyboardInterrupt:
            print('user paused loop')
            break
        except:
            print(sys.exc_info())
            pass
    # print(asin_page)
    return asin_page,current_asin_index


In [ ]:
dfASINList = pd.read_excel(r'Philips Top Products.xlsx',header = 0,engine = 'openpyxl')
asin_list = list(set(dfASINList['asin']))
asin_list

In [104]:
locator = 'div>div>div>h1'
timeout = 10
asin_start_index = 0 #change to asin_current_index to resume loop from the point where it was paused
# asin_start_index = asin_current_index #
asin_details = {} ## comment this line and update asin_details with asin_page before restarting loop
asin_page,asin_current_index = getData(asin_list,asin_start_index,timeout,locator,driver)
asin_details.update(asin_page)
print("asin details", asin_page)

  0%|          | 0/1005 [00:00<?, ?it/s]

NG', 'Description_product': 'Perfect trim with self sharpening stainless steel blades', 'timestamp': '2021-05-26 17:15:58.233255'}, 'feature details': {'timestamp': '2021-05-26 17:15:58.252948'}, 'hl_emi details': {'Highlights': 'Blade Material: Stainless Steel\nTrimming Range: 0.5 - 7 mm\n30 min battery run time\n2 length settings\nGender: Men\nFor Beard & Moustache', 'timestamp': '2021-05-26 17:15:58.305711'}, 'images': [], 'num vids': 0, 'timestamp': '2021-05-26 17:15:58.305711'}, 'PUTFVYC5HETHVDEG': {'product title': 'PHILIPS HD4815/01 pack of 3 800 W Pop Up Toaster  (White)', 'product details': {'Sales Package': '3 toster', 'Brand': 'PHILIPS', 'Model': 'HD4815/01 pack of 3', 'Color': 'White', 'No of Slices': 2, 'Power Consumption': '800 W', 'Service Type': 'Customer needs to carry the product to the nearby Authorized Service Center to get the product repaired or inspected.', 'Warranty Summary': '2 Years Warranty by Philips Pop-up Toaster', 'Covered In Warranty': 'Manufacturing Def

In [105]:
df_FlipkartData = pd.DataFrame.from_dict(asin_page,orient = 'index')

In [124]:
df_FlipkartData.to_csv('philips_data.csv')

In [109]:
df_FlipkartData

,product title,product details,feature details,hl_emi details,images,num vids,timestamp
TMRFSXHRMGTRGSUB,PHILIPS MG 3730 Runtime: 60 min Trimmer for Me...,"{'1 Trimmer set': nan, 'Brand': 'PHILIPS', 'Mo...",{'timestamp': '2021-05-26 16:41:31.216264'},{'Highlights': 'Blade Material: Titanium Coate...,[],0,2021-05-26 16:41:31.263909
IRNFZV2YUBUZ3PTZ,PHILIPS Dry Iron 1000 WATT 1000 W Steam Iron ...,"{'Sales Package': 'IRON', 'Brand': 'PHILIPS', ...",{'timestamp': '2021-05-26 16:41:32.872069'},{'Highlights': 'Steam Iron TEFLON Plate Consum...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-05-26 16:41:32.960085
IRNF5PDZRSABTJPF,PHILIPS HI 114 1000 W Dry Iron (White),"{'Sales Package': 1, 'Brand': 'PHILIPS', 'Mode...",{'timestamp': '2021-05-26 16:41:34.978542'},{'Highlights': 'Dry Iron Golden American Herit...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-05-26 16:41:35.095613
TMRFG4XHFDEBCWFZ,PHILIPS BT3201/15 Runtime: 30 min Trimmer for ...,"{'1 trimmer, user manual, warranty card': nan,...",{'timestamp': '2021-05-26 16:41:36.613321'},{'Highlights': 'Blade Material: Stainless Stee...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-05-26 16:41:36.713799
ICTFZJ2FBDXFH3N8,PHILIPS Viva Collection HD4938/01 2100-Watt In...,"{'Sales Package': '1 INDUCTION', 'Brand': 'PHI...",{'timestamp': '2021-05-26 16:41:38.354163'},{'Highlights': 'Type: Induction Cooktop Workto...,[],0,2021-05-26 16:41:38.428859
...,...,...,...,...,...,...,...
EPRFZ6XESFJHQ5TP,PHILIPS BRE710/00 pack of 2 Cordless Epilator ...,"{'Epilator, Shaving Head, Trimming Comb, Delic...",{'timestamp': '2021-05-26 17:16:07.723137'},{'Highlights': 'For Legs Used For: Dry and Wet...,[],0,2021-05-26 17:16:07.797271
TMRFVW7GNQGT6ZAZ,PHILIPS HC3505/15 Runtime: 0 mins Trimmer for ...,"{'1 Trimmer, Hair Clipper, User Manual, Warran...",{'timestamp': '2021-05-26 17:16:10.046195'},{'Highlights': 'Blade Material: Stainless Stee...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-05-26 17:16:10.117286
MIXEHC24GCZCJS6F,"PHILIPS hl 7756 750 W Mixer Grinder (3 Jars, B...",{'Sales Package': '1 mixer grinder and 3 jars'...,{'timestamp': '2021-05-26 17:16:11.907351'},"{'Total Jars': '3', 'Highlights': '750 W : Hig...",[https://rukminim1.flixcart.com/image/416/416/...,0,2021-05-26 17:16:12.012598
TMRFUFZ3HK9CHZWM,PHILIPS TRIMMER VIRAT KOHLI SIGNATURE LIMITED ...,"{'TRIMMER, CUTTING SHEET': nan, 'Brand': 'PHIL...",{'timestamp': '2021-05-26 17:16:13.328997'},{'Highlights': 'Blade Material: Stainless Stee...,[],0,2021-05-26 17:16:13.373725


In [123]:
df_FlipkartData['product details'][14]

{'Manual, Concentrator, Main Unit': nan,
 'Brand': 'PHILIPS',
 'Model Number': 'HP8100/60',
 'Color': 'Blue',
 'Number of Speed Settings': '2',
 'Number of Heat Settings': '2',
 'Number of Attachments': '4',
 'Attachment Types': 'Concentrator',
 'Nozzle/Concentrator': 'yes',
 'Cold Air Feature': 'No',
 'Wattage': '1000 W',
 'Power Required (Volts)': '220-240',
 'Universal Voltage': 'Yes',
 'Removable Filter': 'Yes',
 'Hanging Loop': 'Yes',
 'Cord Length': '1.5 m',
 'Foldable': 'No',
 'Warranty Summary': '2-year guarantee',
 'Service Type': '2-year guarantee',
 'Covered in Warranty': '2-year guarantee',
 'Not Covered in Warranty': '2-year guarantee',
 'Description_product': 'Compact and ergonomic, this hairdryer benefits from a clever modern design. This results in a dryer that is light and easy to handle yet small enough to store virtually anywhere.This 1000W hairdryer creates the optimum level of airflow and gentle drying power, for beautiful results every day.The rubberised hook is l

In [ ]:
asin_details.update(asin_page)

In [ ]:
asin_details

In [ ]:
img_list,num_vids = getCatalogContent()
print("Images",img_list,"Number of videos",num_vids)